<a href="https://colab.research.google.com/github/ranieri-unimi/faces-malchiodi-2022/blob/main/faces.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### run once

In [1]:
!pip install pyspark
!pip install findspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import os
os.environ["KAGGLE_USERNAME"] = 'ranieriunimi'
os.environ["KAGGLE_KEY"] = str(hex(232307088475198570779809482024044346960))[2:]

In [3]:
ref = 'bwandowando/ukraine-russian-crisis-twitter-dataset-1-2-m-rows'
!mkdir datasets
!kaggle datasets download $ref --unzip -p ./datasets

mkdir: cannot create directory ‘datasets’: File exists
100% 9.34G/9.35G [01:28<00:00, 145MB/s]
100% 9.35G/9.35G [01:28<00:00, 113MB/s]


In [4]:
import nltk
nltk.download('all');

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package basque_grammars is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Pac

## ukraine

In [5]:
SAMPLE_SIZE = 1024

In [6]:
import numpy as np
import pandas as pd
import csv
import re
import string
import random

In [7]:
import pyspark
import findspark

In [8]:
# load dataset 
filename = r"./datasets/UkraineCombinedTweetsDeduped20220227-131611.csv.gzip"
pd.set_option("display.max_columns", None)
df = pd.read_csv(filename, compression='gzip', index_col=0, encoding='utf-8', quoting=csv.QUOTE_ALL)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


## data cleaning

In [9]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

eng_sw = set(stopwords.words("english"))

In [10]:
df = df[df.language == 'en'].text.tolist()

In [11]:
if SAMPLE_SIZE:
    df = random.sample(df, SAMPLE_SIZE)

In [12]:
# https://www.pluralsight.com/guides/building-a-twitter-sentiment-analysis-in-python

def preprocess_tweet_text(tweet):
    index, tweet = tweet

    tweet = tweet.lower()

    # cleanings 🧹

    # urls
    tweet = re.sub(r"http\S+|www\S+|https\S+", '', tweet, flags=re.MULTILINE)

    # # @ and #
    # tweet = re.sub(r'\@\w+|\#','', tweet)

    # punctuations
    # tweet = tweet.translate(str.maketrans('', '', string.punctuation))
    tweet = tweet.translate(str.maketrans(string.punctuation+'…’”“', ' '*(len(string.punctuation)+4)))  # puntctuation to spaces


    # tweet_tokens = word_tokenize(tweet)

    # # emojitter
    # wrds = [e for word in tweet_tokens for e in re.findall(r"(\w+|[^\w ]+)", word)]
    # # TODO split also emoji-goups

    # # stopwords
    # filtered_words = [w for w in wrds if not w in eng_sw]
    
    # # stemmatize
    # ps = PorterStemmer()
    # stemmed_words = [ps.stem(w) for w in filtered_words]

    # # lemmatize
    # lemmatizer = WordNetLemmatizer()
    # lemma_words = [str(lemmatizer.lemmatize(w, pos='a')) for w in stemmed_words]
  
    # return (index, lemma_words)

    tweet = ' '.join(tweet.strip().split())

    return (index, tweet)

In [13]:
#lang_hist = {l:df[df.language == l].size for l in df.language.unique()}

## hadoop instance

In [14]:
# import findspark
# findspark.init("spark-3.1.1-bin-hadoop3.2") # SPARK_HOME
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = spark.sparkContext

In [15]:
rdd = sc.parallelize(enumerate(df))

In [16]:
rdd.take(5)

[(0,
  '#Anonymous is currently involved in operations against the Russian Federation. Our operations are targeting the Russian government and those close to the Russian Government- We are at War'),
 (1,
  'Paris now🇺🇦 \n#PrayForUkraine #UkraineUnderAttack #UkraineRussia #UkraineWar https://t.co/FgejkCDmLo'),
 (2,
  'Civilians in apartment exchanging fire with Russian Soldiers.\n\n#Ukraine #UkraineInvasion #UkraineRussia #RussiaInvadesUkraine #RussiaInvasion https://t.co/bJpIwituZZ'),
 (3,
  "If you don't know how to help out, start by exposing and reporting Russian disinformation, it's an important job during these times. #Anonymous"),
 (4,
  '#Anonymous is continuing its operations against the Russian Federation. Our operations are targeting the Russian government and those close to the Russian Government- We are at War ! We are united with #ukraine 🇺🇦 Mr Putin ! You have been warned - Expect MORE!')]

In [17]:
dataPipe = rdd.map(preprocess_tweet_text)

In [18]:
dataPipe.take(5)

[(0,
  'anonymous is currently involved in operations against the russian federation our operations are targeting the russian government and those close to the russian government we are at war'),
 (1, 'paris now🇺🇦 prayforukraine ukraineunderattack ukrainerussia ukrainewar'),
 (2,
  'civilians in apartment exchanging fire with russian soldiers ukraine ukraineinvasion ukrainerussia russiainvadesukraine russiainvasion'),
 (3,
  'if you don t know how to help out start by exposing and reporting russian disinformation it s an important job during these times anonymous'),
 (4,
  'anonymous is continuing its operations against the russian federation our operations are targeting the russian government and those close to the russian government we are at war we are united with ukraine 🇺🇦 mr putin you have been warned expect more')]

In [19]:
# indexing = dataset.flatMap(lambda _, v : [(e,e) for e in v]).reduceByKey(lambda k, v : k)

### foos

In [20]:
def is_sub(sub, lst) : return all(e in lst for e in sub)
def add(a, b) : return a+b
def splat(t): return tuple(sorted(list(j for i in t for j in (i if isinstance(i, tuple) else (i,)))))
def doubled(t): return len(set(t)) == len(t)

In [21]:
# nen_rdd = sc.parallelize(df)

In [22]:
# elemListPipe = nen_rdd.flatMap(lambda word_list : word_list).distinct()
# elem_index = sc.parallelize(enumerate(elemListPipe.collect()))

### shingles

In [37]:
SH_LEN = 9

In [24]:
shigles = dataPipe.flatMap(lambda x : [(x[-1][i:i+SH_LEN], x[0]) for i in range(0, len(x[-1])-SH_LEN)]).distinct()

In [25]:
# import random
# HSEED = 42
# random.seed(HSEED)

In [26]:
# HBYTES = 4
# HMAX = 2**(8*HBYTES)
# SBOX = list(range(HMAX))
# random.shuffle(SBOX)

In [27]:
# def phash(message):
#   hash = len(message) % HMAX
#   for i in message:
#     hash = SBOX[(hash + ord(i)) % HMAX]
#   return hash

In [38]:
import binascii

def shingling(x):
  k, v = x
  shingles = []
  for i in range(len(v) - SH_LEN):
    shingle = v[i:i+SH_LEN]
    hshingle = binascii.crc32(bytes(shingle, 'utf-8'))
    shingles.append( (hshingle, k) )
  return shingles

In [39]:
 dataPipe.flatMap(shingling).distinct().take(10)

[(3258052014, 0),
 (3753018060, 0),
 (3759533014, 0),
 (641121970, 0),
 (13803814, 0),
 (3615978982, 0),
 (2279638598, 0),
 (56825562, 0),
 (1531410548, 0),
 (194541752, 0)]

In [ ]:
# HBYTES = 4
# HMAX = 2**(8*HBYTES)
# SBOX = list(range(HMAX))